## Residual Network 구현 및 학습

In [1]:
import tensorflow as tf
import numpy as np

## 데이터셋 준비


In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## Residual Unit 구현

In [3]:
class ResidualUnit(tf.keras.Model):
    def __init__(self, filter_in, filter_out, kernel_size):
        super(ResidualUnit, self).__init__()

        # Pre-Activation : BN-relu-conv
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        
        # Identity mapping
        if filter_in == filter_out:
            self.identity = lambda x: x
        else:
            # convolution을 이용해 filter 수를 맞춰준다.
            self.identity = tf.keras.layers.Conv2D(filter_out, (1,1), padding='same')

    def call(self, x, training=False, mask=None):
        h = self.bn1(x, training=training)
        h = tf.nn.relu(h)
        h = self.conv1(h)
        
        h = self.bn2(h, training=training)
        h = tf.nn.relu(h)
        h = self.conv2(h)
        
        return self.identity(x) + h

## Residual Layer 구현

In [10]:
class ResnetLayer(tf.keras.Model):
    def __init__(self, filter_in, filters, kernel_size):
        super(ResnetLayer, self).__init__()
        
        self.sequence = list()
        
        for f_in, f_out in zip([filter_in] + list(filters), filters):  # [8,16,16], [16,16]
            print(f_in, f_out)
            self.sequence.append(ResidualUnit(f_in, f_out, kernel_size))

    def call(self, x, training=False, mask=None):
        for unit in self.sequence:
            x = unit(x, training=training)
            
        return x

## 모델 정의

In [11]:
class ResNet(tf.keras.Model):
    def __init__(self):
        super(ResNet, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(8, (3, 3), padding='same', activation='relu') # 28x28x8
        
        self.res1 = ResnetLayer(8, (16, 16), (3, 3))   # 28x28x16 : BN-relu-conv-BN-relu-conv
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2)) # 14x14x16
        
        self.res2 = ResnetLayer(16, (32, 32), (3, 3))  # 14x14x32
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2)) # 7x7x32
        
        self.res3 = ResnetLayer(32, (64, 64), (3, 3))  # 7x7x64
        
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')
        
    def call(self, x, training=False, mask=None):
        x = self.conv1(x)
        
        x = self.res1(x, training=training)
        x = self.pool1(x)
        
        x = self.res2(x, training=training)
        x = self.pool2(x)
        
        x = self.res3(x, training=training)
        x = self.flatten(x)
        x = self.dense1(x)
        
        return self.dense2(x)

In [12]:
# 모델 구조 확인
model = ResNet()
model.build(input_shape=(None, 28,28,1))
model.summary()

8 16
16 16
16 32
32 32
32 64
64 64
Model: "res_net_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           multiple                  80        
_________________________________________________________________
resnet_layer_6 (ResnetLayer) multiple                  8496      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 multiple                  0         
_________________________________________________________________
resnet_layer_7 (ResnetLayer) multiple                  33376     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 multiple                  0         
_________________________________________________________________
resnet_layer_8 (ResnetLayer) multiple                  132288    
_________________________________________________________________
flatten_2 (Flatten)   

## 학습, 테스트 루프 정의

In [6]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [7]:
# Create model
model = ResNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [8]:
model.build(input_shape=(None, 28,28,1))
model.summary()

Model: "res_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  80        
_________________________________________________________________
resnet_layer (ResnetLayer)   multiple                  8496      
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
resnet_layer_1 (ResnetLayer) multiple                  33376     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
resnet_layer_2 (ResnetLayer) multiple                  132288    
_________________________________________________________________
flatten (Flatten)            multiple                  0   

## 학습 루프 동작

In [9]:
EPOCHS = 10

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {:.3f}, Accuracy: {:.3f}, Test Loss: {:.3f}, Test Accuracy: {:.3f}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result(),
                          test_loss.result(),
                          test_accuracy.result()))
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.146, Accuracy: 0.960, Test Loss: 0.063, Test Accuracy: 0.982
Epoch 2, Loss: 0.067, Accuracy: 0.982, Test Loss: 0.061, Test Accuracy: 0.984
Epoch 3, Loss: 0.048, Accuracy: 0.987, Test Loss: 0.075, Test Accuracy: 0.976
Epoch 4, Loss: 0.042, Accuracy: 0.988, Test Loss: 0.048, Test Accuracy: 0.986
Epoch 5, Loss: 0.033, Accuracy: 0.991, Test Loss: 0.043, Test Accuracy: 0.988
Epoch 6, Loss: 0.029, Accuracy: 0.992, Test Loss: 0.041, Test Accuracy: 0.987
Epoch 7, Loss: 0.028, Accuracy: 0.992, Test Loss: 0.028, Test Accuracy: 0.991
Epoch 8, Loss: 0.024, Accuracy: 0.993, Test Loss: 0.046, Test Accuracy: 0.987
Epoch 9, Loss: 0.022, Accuracy: 0.994, Test Loss: 0.049, Test Accuracy: 0.987
Epoch 10, Loss: 0.019, Accuracy: 0.995, Test Loss: 0.035, Test Accuracy: 0.992
